In [1]:
import requests as r
import pandas as pd
import numpy as np

url = "https://www.transfermarkt.com/liga-mx-apertura/startseite/wettbewerb/MEXA/plus/?saison_id=2010"

header = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'}

response = r.get(url, headers=header)

In [2]:
cont = r.get(url, headers = header).text
t = pd.read_html(cont, attrs = {'class': 'items'})[0]

In [3]:
t.drop(columns=['Club', 'Club.1', 'Players playing abroad', 'name.1', 'Total MV', 'ø MV'], inplace= True)
t.rename({'name':'Club', 'Squad':'Edad Promedio Jugador', 'ø age':'Jugadores Extranjeros', 'Total market value':'Valor Promedio de Jugador(€)', 'ø market value':'Valor Equipo(€)'}, axis = 1, inplace = True)

In [4]:
t.index = np.arange(1,len(t)+1)
df_list = []

In [5]:
for i in range(12, 20):
    cont = r.get(f"https://www.transfermarkt.com/liga-mx-apertura/startseite/wettbewerb/MEXA/plus/?saison_id=20{i}", headers = header).content
    t = pd.read_html(cont, attrs = {'class': 'items'})[0]
    t.drop(columns=['Club', 'Club.1', 'Players playing abroad', 'name.1', 'Total MV', 'ø MV'], inplace= True)
    t.rename({'name':'Club', 'Squad':'Edad Promedio Jugador', 'ø age':'Jugadores Extranjeros', 'Total market value':'Valor Promedio de Jugador(€)', 'ø market value':'Valor Equipo(€)'}, axis = 1, inplace = True)
    t.index = np.arange(1,len(t)+1)
    df_list.append(t)
    print(f'Temporada 20{i} Done!')

Temporada 2012 Done!
Temporada 2013 Done!
Temporada 2014 Done!
Temporada 2015 Done!
Temporada 2016 Done!
Temporada 2017 Done!
Temporada 2018 Done!
Temporada 2019 Done!


In [6]:
with pd.ExcelWriter('Valores_Equipos_LigaMX.xlsx') as writer:
    temp = 2012
    for df in df_list:
        df.to_excel(writer, sheet_name=f'Temp {temp}')
        temp += 1
        
print("[+] Excel Generado!")

[+] Excel Generado!


In [7]:
df_list

[             Club  Edad Promedio Jugador  Jugadores Extranjeros  \
 1     Tigres UANL                    249                      9   
 2      CF Pachuca                    240                     12   
 3       Monterrey                    244                      6   
 4         América                    234                      8   
 5    CD Cruz Azul                    241                      5   
 6          Chivas                    225                      0   
 7         Morelia                    247                      7   
 8   Santos Laguna                    246                      9   
 9        San Luis                    254                     10   
 10     UNAM Pumas                    232                      7   
 11         Toluca                    256                      5   
 12       Jaguares                    232                      7   
 13          Atlas                    241                      8   
 14   Querétaro FC                    249       

In [8]:
def sum_dfs(df_list):
    tables = []
    for table in df_list:
        table.sort_values('Club', inplace = True)
        tables.append(table)         
    return tables

In [9]:
sum_dfs(df_list)

[             Club  Edad Promedio Jugador  Jugadores Extranjeros  \
 4         América                    234                      8   
 16        Atlante                    246                      9   
 13          Atlas                    241                      8   
 5    CD Cruz Azul                    241                      5   
 2      CF Pachuca                    240                     12   
 6          Chivas                    225                      0   
 17   Club Tijuana                    255                     11   
 12       Jaguares                    232                      7   
 18           León                    244                      7   
 3       Monterrey                    244                      6   
 7         Morelia                    247                      7   
 15      Puebla FC                    260                      9   
 14   Querétaro FC                    249                     11   
 9        San Luis                    254       

In [10]:
result = pd.concat(df_list)

In [11]:
df_list[0].head()

,Club,Edad Promedio Jugador,Jugadores Extranjeros,Valor Promedio de Jugador(€),Valor Equipo(€)
4,América,234,8,957 K €,"39,25 mil. €"
16,Atlante,246,9,510 K €,"23,95 mil. €"
13,Atlas,241,8,548 K €,"26,30 mil. €"
5,CD Cruz Azul,241,5,951 K €,"37,10 mil. €"
2,CF Pachuca,240,12,969 K €,"43,63 mil. €"


In [13]:
import re

In [14]:
def clean_euros(x):
    if "mil. €" in x: return int(re.sub(" mil. €",'000000', x).replace(',',''))
    if "K €" in x: return int(re.sub(' K €','000', x))


In [15]:
df_list[-1] = df_list[-1].dropna()

In [16]:
for df in df_list:
    df['Jugadores Extranjeros'] = df['Jugadores Extranjeros'].apply(lambda x: int(x))
    df['Valor Promedio de Jugador(€)'] = df['Valor Promedio de Jugador(€)'].apply(clean_euros)
    df['Valor Equipo(€)'] = df['Valor Equipo(€)'].apply(clean_euros)
    df['Edad Promedio Jugador'] = df['Edad Promedio Jugador'].apply(lambda x: x/80)
    

/home/jelv/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/jelv/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/jelv/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [17]:
df_list

[             Club  Edad Promedio Jugador  Jugadores Extranjeros  \
 4         América                 2.9250                      8   
 16        Atlante                 3.0750                      9   
 13          Atlas                 3.0125                      8   
 5    CD Cruz Azul                 3.0125                      5   
 2      CF Pachuca                 3.0000                     12   
 6          Chivas                 2.8125                      0   
 17   Club Tijuana                 3.1875                     11   
 12       Jaguares                 2.9000                      7   
 18           León                 3.0500                      7   
 3       Monterrey                 3.0500                      6   
 7         Morelia                 3.0875                      7   
 15      Puebla FC                 3.2500                      9   
 14   Querétaro FC                 3.1125                     11   
 9        San Luis                 3.1750       

In [18]:
result = pd.concat(df_list)

In [19]:
final_table = result.groupby(['Club']).sum()

In [20]:
final_table

,Edad Promedio Jugador,Jugadores Extranjeros,Valor Promedio de Jugador(€),Valor Equipo(€)
Club,,,,
América,24.4750,100,8.338360e+08,3.725900e+10
Atlante,6.3750,20,1.171000e+06,5.567000e+09
Atlas,24.5500,96,1.264100e+08,2.482600e+10
CD Cruz Azul,26.2125,91,9.459510e+08,3.725300e+10
CF Pachuca,24.1250,109,3.349170e+08,3.362900e+10
Chiapas FC,12.8625,59,2.763000e+06,1.324800e+10
Chivas,24.1375,1,3.995140e+08,3.126500e+10
Club Tijuana,25.0250,151,3.443280e+08,2.717800e+10
Dorados de Sin.,3.2375,16,6.220000e+05,3.233000e+09


## Vemos que del 2012 a la fecha los equipos con mas valor en el mercado son los de MTY
## Seguido de los de la CDMX dejando aun lado PUMAS

In [21]:
top_valuable_teams = final_table.sort_values('Valor Equipo(€)', ascending=False)
top_valuable_teams[:10]

,Edad Promedio Jugador,Jugadores Extranjeros,Valor Promedio de Jugador(€),Valor Equipo(€)
Club,,,,
Tigres UANL,25.6500,105,1.208938e+09,4.993800e+10
Monterrey,24.7750,99,1.242000e+09,4.564000e+10
América,24.4750,100,8.338360e+08,3.725900e+10
CD Cruz Azul,26.2125,91,9.459510e+08,3.725300e+10
CF Pachuca,24.1250,109,3.349170e+08,3.362900e+10
Santos Laguna,24.2750,87,7.558070e+08,3.231300e+10
Chivas,24.1375,1,3.995140e+08,3.126500e+10
León,25.1750,102,2.807920e+08,2.838600e+10
Club Tijuana,25.0250,151,3.443280e+08,2.717800e+10


# Equipo con mas extranjeros desde 2012
#### Vemos que Tijuana es quien mas jugadores extranjeros ha tenido desde 2012 siendo el 8 mejor de la liga
#### Es necesario analizar este outlier (Jugadores doble nacionalidad?)

In [22]:
top_jug_extr = final_table.sort_values('Jugadores Extranjeros', ascending=False)
top_jug_extr

,Edad Promedio Jugador,Jugadores Extranjeros,Valor Promedio de Jugador(€),Valor Equipo(€)
Club,,,,
Club Tijuana,25.0250,151,3.443280e+08,2.717800e+10
CF Pachuca,24.1250,109,3.349170e+08,3.362900e+10
Tiburones Rojos,23.0250,106,4.311000e+06,1.963800e+10
Tigres UANL,25.6500,105,1.208938e+09,4.993800e+10
Puebla FC,26.7000,104,5.330000e+06,2.239000e+10
León,25.1750,102,2.807920e+08,2.838600e+10
Querétaro FC,25.6375,102,1.168400e+08,2.403500e+10
América,24.4750,100,8.338360e+08,3.725900e+10
Monterrey,24.7750,99,1.242000e+09,4.564000e+10


# Puebla es quien tiene jugadores mas viejos en promedio desde 2012
#### El rango de edad promedio es de los 24 a 26 
#### PENDIENTE: Es importante realizar un promedio con los equipos quienes no han estados todos los torneos desde 2012 para tener mejor analisis

In [23]:
top_edad = final_table.sort_values('Edad Promedio Jugador', ascending=False)
top_edad

,Edad Promedio Jugador,Jugadores Extranjeros,Valor Promedio de Jugador(€),Valor Equipo(€)
Club,,,,
Puebla FC,26.7000,104,5.330000e+06,2.239000e+10
CD Cruz Azul,26.2125,91,9.459510e+08,3.725300e+10
Toluca,26.0000,80,3.321390e+08,2.639100e+10
Tigres UANL,25.6500,105,1.208938e+09,4.993800e+10
Querétaro FC,25.6375,102,1.168400e+08,2.403500e+10
Morelia,25.4750,94,1.050950e+08,2.563800e+10
León,25.1750,102,2.807920e+08,2.838600e+10
Club Tijuana,25.0250,151,3.443280e+08,2.717800e+10
Monterrey,24.7750,99,1.242000e+09,4.564000e+10


# Tomando los top 10 equipos, quien tiene jugadores mas jovenes aparentemente es el Veracruz pero en realidad es Pachuca

### Pendiente por ajustar el caso de Veracruz y demas equipos con menor cantidad de juegos, pues esto solo es una sumatoria


In [24]:
top_edad_joven = final_table.sort_values('Edad Promedio Jugador', ascending=True)
top_edad_joven = top_edad_joven[top_edad_joven['Edad Promedio Jugador'] > 19]

In [25]:
top_edad_joven[:10]

,Edad Promedio Jugador,Jugadores Extranjeros,Valor Promedio de Jugador(€),Valor Equipo(€)
Club,,,,
Tiburones Rojos,23.0250,106,4.311000e+06,1.963800e+10
CF Pachuca,24.1250,109,3.349170e+08,3.362900e+10
Chivas,24.1375,1,3.995140e+08,3.126500e+10
Santos Laguna,24.2750,87,7.558070e+08,3.231300e+10
UNAM Pumas,24.3625,77,1.482870e+08,2.563400e+10
América,24.4750,100,8.338360e+08,3.725900e+10
Atlas,24.5500,96,1.264100e+08,2.482600e+10
Monterrey,24.7750,99,1.242000e+09,4.564000e+10
Club Tijuana,25.0250,151,3.443280e+08,2.717800e+10


#### Vemos que Rayados es quien tiene los jugadores mas caros en promedio, pero la plantila mas cara es de Tigres
#### Lo que nos hace suponer que existe uno o dos jugadores en Tigres outliers de valor a comparacion del resto del equipo
#### por lo que este unico (o dos) jugador eleva el valor de la plantilla entera
#### Pendiente: Hipotesis a comprobar --> Es Gignac es el outlier de Tigres y quien aumenta el valor economico de la plantilla

In [26]:
top_valor_jug_prom = final_table.sort_values('Valor Promedio de Jugador(€)', ascending=False)
top_valor_jug_prom[:10]

,Edad Promedio Jugador,Jugadores Extranjeros,Valor Promedio de Jugador(€),Valor Equipo(€)
Club,,,,
Monterrey,24.7750,99,1.242000e+09,4.564000e+10
Tigres UANL,25.6500,105,1.208938e+09,4.993800e+10
CD Cruz Azul,26.2125,91,9.459510e+08,3.725300e+10
América,24.4750,100,8.338360e+08,3.725900e+10
Santos Laguna,24.2750,87,7.558070e+08,3.231300e+10
Chivas,24.1375,1,3.995140e+08,3.126500e+10
Club Tijuana,25.0250,151,3.443280e+08,2.717800e+10
CF Pachuca,24.1250,109,3.349170e+08,3.362900e+10
Toluca,26.0000,80,3.321390e+08,2.639100e+10
